# Hey There!

Welcome to this new and fresh section. This section covers the "Automatic Call to the Function" functionality. 
<br>—<br>
### Background
Till now, as we have used the "Static" data, a data which is neither growing or shrinking... but stays the same, now it is the time to get <font color="aayush"><b>LIVE</b></font> data. Our script will fetch the data from the server and instantly will present in front of us in the form of some cool viz. 

### What to expect
Here in this notebook, I will demonstrate **the problem**, **the idea** and then **the solution**. All step by step as its being a new concept. If you've come to refresh the mind and want to quickly get to the point what is to be used, we are going to use `dcc.Interval()` function to repeat task after some intervals.

Let's go Aayush ∞

###### 

# <center> The Task </center>

We need the script *to remember* the refresh of the page (as the example goes in the course), so when we refresh the page, we "*kind of*" get the new data of how many times the page has been refreshed and then we will print on the screen. So, let's see how.

#### <center> . . .

In [2]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input

# <center> The Problem </center>

In [5]:
app = dash.Dash()

n = 0
n += 1
app.layout = html.Div(children=[
                html.H1(f"This page has been clicked for {n} times!")
            ])

app.run_server()

See ↑ nothing changed on the refresh, because the value is not updated again.

***HINT?*** `app.layout` is the page itself which gets executed fresh, everytime when we run the page. So if there is any mechanism which can give new n+1 value to layout when the page is refreshed, we can get the job done.

So, let's try this ↓

# <center> The Idea </center>

In [12]:
app = dash.Dash()

n = 0
def update_n():
    global n
    n += 1
    layout = html.Div(children=[
                        html.H1(f"This page has been clicked for {n} times!")
                        ])
    return layout

app.layout = update_n()
app.run_server()

Shit! It didn't work either.<br>
- **The main idea** was that, we thought, `app.layout` is a `function` which runs again, and again when we refresh the page, but that isn't true. Because if that was the case, then the implementation above should have worked. 

- **So, what now?** there is a little change instead of writing <code><strike>app.layout = update_n()</strike></code> we wanna write → `app.layout = update_n`.<br>
I mean really! Just remove the paranthesis!

- **What is happening in there?** Our idea was *kind of* right before of thinking the `app.layout` as the function but that was not entierly correct. The correct thing is, *what `app.layout` stores, is reusable or re-callable*. So instead of storing the output of the function with calling `()`, we just store the function which can be called. That's it.

# <center> The Solution </center>

In [14]:
app = dash.Dash()

n = 0
def update_n():
    global n
    n += 1
    layout = html.Div(children=[
                        html.H1(f"This page has been clicked for {n} times!")
                        ])
    return layout

             ### POI ↓ POI ###
app.layout = update_n
app.run_server()

Easy! Isn't it?

If you want to check, the internal representation is...
```python
              # This ↓ gets called
layout_value = self._layout_value()
self.css._update_layout(layout_value)
```

# 

# Now The Real Hero

Till now, I wanted to give the idea of "how to get new data". So, now you will see the actual thing to go when we need to have the realtime data. That is with (let me make it in big fonts)
# `dcc.Interval()`

Without the prior explanation let's look at the code first.

In [9]:
app = dash.Dash()

app.layout = html.Div(children=[
                html.H1(id="label"),
                dcc.Interval(interval=1000,          ### POI ← POI ###
                             n_intervals=10,
                             id="automatic_caller")
])

@app.callback(Output("label", "children"),
             [Input("automatic_caller", "n_intervals")])
def get_count(n_intervals):
    return f"This page has been refreshed {n_intervals} times!"

app.run_server()

Amazingly works!

**Syntax Analysis**
```python
app.layout = html.Div(children=[
                html.H1(id="label"),
                dcc.Interval(interval=1000, # ...(1)
                             n_intervals=10, # ...(2)
                             id="automatic_caller")
])

```

First thing is, `dcc.Interval` is passed directly as a dcc object like any other.  
Second thing is, when you just define it like that, it will get call automatically after that many times.  
—<br>
(1) `interval` suggests after how much time (milliseconds) you wanna call a function  
(2) `n_intervals` it has to be set, and if set to some other number like 100 then the counting will start from 100, 101, 102... if we decide not to set, it will be **None** and the sequence will be None, 1, 2...

 <u><b>Q</b></u> What if there are ***multiple*** `dcc.Intervals()`? <br>
 <u><b>A</b></u> They will call and affect different functions individually. Let's see practically below ↓
 

### 1. All *Intervals* attatched to some function

In [12]:
app = dash.Dash()

app.layout = html.Div(children=[
                html.H1(id="label_1"),
                html.H1(id="label_2"),
                html.H1(id="label_3"),
                
                dcc.Interval(interval=2000,
                             n_intervals=0,
                             id="caller_1"),
                dcc.Interval(interval=1000,
                             n_intervals=0,
                             id="caller_2"),
                dcc.Interval(interval=500,
                             n_intervals=0,
                             id="caller_3")
])

# 1 Label
@app.callback(Output("label_1", "children"),
             [Input("caller_1", "n_intervals")])
def get_count_1(n_intervals):
    return f"This page has been refreshed {n_intervals} times AFTER 2 SECONDS GAP!"


# 2 Label
@app.callback(Output("label_2", "children"),
             [Input("caller_2", "n_intervals")])
def get_count_2(n_intervals):
    return f"This page has been refreshed {n_intervals} times AFTER 1 SECOND GAP!"


# 3 Label
@app.callback(Output("label_3", "children"),
             [Input("caller_3", "n_intervals")])
def get_count_3(n_intervals):
    return f"This page has been refreshed {n_intervals} times! AFTER 0.5 SECOND GAP!"

app.run_server()

See it works!

### 2. NOT All *Intervals* attatched to some function

In [14]:
app = dash.Dash()

app.layout = html.Div(children=[
                html.H1(id="label_1"),
                html.H1(id="label_2"),
                html.H1(id="label_3"),
                
                dcc.Interval(interval=2000,
                             n_intervals=0,
                             id="caller_1"),
                dcc.Interval(interval=1000,
                             n_intervals=0,
                             id="caller_2"),
                dcc.Interval(interval=500,
                             n_intervals=0,
                             id="caller_3")
])

# 1 Label
@app.callback(Output("label_1", "children"),
             [Input("caller_1", "n_intervals")])
def get_count_1(n_intervals):
    return f"This page has been refreshed {n_intervals} times AFTER 2 SECONDS GAP!"


### UN-ATTACHING 2nd INTERVAL ###
# ----------------------------- #
# @app.callback(Output("label_2", "children"),
#              [Input("caller_2", "n_intervals")])
# def get_count_2(n_intervals):
#     return f"This page has been refreshed {n_intervals} times AFTER 1 SECOND GAP!"
# ----------------------------- #
### UN-ATTACHING 2nd INTERVAL ###


# 3 Label
@app.callback(Output("label_3", "children"),
             [Input("caller_3", "n_intervals")])
def get_count_3(n_intervals):
    return f"This page has been refreshed {n_intervals} times! AFTER 0.5 SECOND GAP!"

app.run_server()

Still works! Just except the `2nd` label **is not shown.**

# 

Great! Now the basics of `dcc.Interval` are clear, we can move to...
# Next step,
When we will get the benefit of this functionality to get new and LIVE data and update our Dashboard!